In [1]:
import os
import joblib
import numpy as np
import pandas as pd
from scipy import stats
import pingouin as pg

In [2]:
os.getcwd()

'C:\\Users\\hdsce\\Documents\\PythonDataAnalysis2\\live'

In [3]:
os.chdir('../data')

In [4]:
os.listdir()

['Used_Cars_Price.csv', 'Used_Cars_Price.xlsx', 'Used_Cars_Price.z']

In [5]:
df = joblib.load('Used_Cars_Price.z')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1268 entries, 0 to 1267
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      1268 non-null   int64 
 1   Age        1268 non-null   int64 
 2   KM         1268 non-null   int64 
 3   FuelType   1268 non-null   object
 4   HP         1268 non-null   int64 
 5   MetColor   1268 non-null   object
 6   Automatic  1268 non-null   object
 7   CC         1268 non-null   int64 
 8   Doors      1268 non-null   int64 
 9   Weight     1268 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 99.2+ KB


In [7]:
df['Age'].cov(df['Price'])

-22157.692905818418

In [10]:
pd.set_option('display.float_format', lambda x: f'{x:.4f}')

In [11]:
df.cov(numeric_only = True)

,Price,Age,KM,HP,CC,Doors,Weight
Price,4117236.3700,-22157.6929,-37441074.7735,5899.6856,18808.7476,317.0113,15155.2218
Age,-22157.6929,187.9416,171187.1245,-7.9903,-213.6068,-1.1809,-98.2257
KM,-37441074.7735,171187.1245,1285819949.2532,-155996.1200,2557934.8056,595.8727,359258.4576
HP,5899.6856,-7.9903,-155996.1200,171.6076,-46.6858,1.4965,-39.0906
CC,18808.7476,-213.6068,2557934.8056,-46.6858,33729.9986,23.8543,5001.2930
Doors,317.0113,-1.1809,595.8727,1.4965,23.8543,0.8998,13.2114
Weight,15155.2218,-98.2257,359258.4576,-39.0906,5001.2930,13.2114,1145.0785


In [12]:
df['Age'].corr(df['Price'])

-0.7965447290519272

In [13]:
df.corr(numeric_only = True)

,Price,Age,KM,HP,CC,Doors,Weight
Price,1.0000,-0.7965,-0.5146,0.2220,0.0505,0.1647,0.2207
Age,-0.7965,1.0000,0.3482,-0.0445,-0.0848,-0.0908,-0.2117
KM,-0.5146,0.3482,1.0000,-0.3321,0.3884,0.0175,0.2961
HP,0.2220,-0.0445,-0.3321,1.0000,-0.0194,0.1204,-0.0882
CC,0.0505,-0.0848,0.3884,-0.0194,1.0000,0.1369,0.8047
Doors,0.1647,-0.0908,0.0175,0.1204,0.1369,1.0000,0.4116
Weight,0.2207,-0.2117,0.2961,-0.0882,0.8047,0.4116,1.0000


In [14]:
pg.corr(x = df['Age'], y = df['Price'])

,n,r,CI95%,p-val,BF10,power
pearson,1268,-0.7965,"[-0.82, -0.78]",0.0000,6.169e+274,1.0000


In [15]:
pg.corr(x = df['Age'], y = df['Price'])['p-val']

pearson   0.0000
Name: p-val, dtype: float64

In [16]:
corr = lambda x: pg.corr(x = x, y = df['Price'])['p-val']

In [17]:
corr(x = df['Age'])

pearson   0.0000
Name: p-val, dtype: float64

In [18]:
df.dtypes

Price         int64
Age           int64
KM            int64
FuelType     object
HP            int64
MetColor     object
Automatic    object
CC            int64
Doors         int64
Weight        int64
dtype: object

In [21]:
locs = (df.dtypes == 'int64') | (df.dtypes == 'float64')

In [22]:
df.loc[:, locs]

,Price,Age,KM,HP,CC,Doors,Weight
0,13500,23,46986,90,2000,3,1165
1,13750,23,72937,90,2000,3,1165
2,13950,24,41711,90,2000,3,1165
3,14950,26,48000,90,2000,3,1165
4,13750,30,38500,90,2000,3,1170
...,...,...,...,...,...,...,...
1263,15950,25,25000,97,1400,3,1100
1264,15500,25,49163,110,1600,5,1165
1265,15950,25,29719,97,1400,5,1110
1266,15950,19,51884,97,1400,3,1100


In [32]:
# df.select_dtypes(include = [int, float])
df.select_dtypes(exclude = [object])

,Price,Age,KM,HP,CC,Doors,Weight
0,13500,23,46986,90,2000,3,1165
1,13750,23,72937,90,2000,3,1165
2,13950,24,41711,90,2000,3,1165
3,14950,26,48000,90,2000,3,1165
4,13750,30,38500,90,2000,3,1170
...,...,...,...,...,...,...,...
1263,15950,25,25000,97,1400,3,1100
1264,15500,25,49163,110,1600,5,1165
1265,15950,25,29719,97,1400,5,1110
1266,15950,19,51884,97,1400,3,1100


In [26]:
df.loc[:, locs].apply(func = lambda x: pg.corr(x = x, y = df['Price'])['r'])

,Price,Age,KM,HP,CC,Doors,Weight
pearson,1.0000,-0.7965,-0.5146,0.2220,0.0505,0.1647,0.2207


In [25]:
df.loc[:, locs].apply(func = corr, axis = 0)

,Price,Age,KM,HP,CC,Doors,Weight
pearson,0.0000,0.0000,0.0000,0.0000,0.0724,0.0000,0.0000


In [34]:
result = df.loc[:, locs].apply(func = corr, axis = 0).lt(0.05)
result

,Price,Age,KM,HP,CC,Doors,Weight
pearson,True,True,True,True,False,True,True


In [35]:
result.iloc[0]

Price      True
Age        True
KM         True
HP         True
CC        False
Doors      True
Weight     True
Name: pearson, dtype: bool

In [38]:
locs = np.where(~result.iloc[0])
locs[0]

array([4], dtype=int64)

In [39]:
result.columns[locs[0]]

Index(['CC'], dtype='object')

In [40]:
pg.normality(data = df, dv = 'Price', group = 'MetColor')

,W,pval,normal
MetColor,,,
1,0.9748,0.0000,False
0,0.9881,0.0014,False


In [41]:
pg.homoscedasticity(data = df, dv = 'Price', group = 'MetColor')

,W,pval,equal_var
levene,5.7613,0.0165,False


In [42]:
df.head()

,Price,Age,KM,FuelType,HP,MetColor,Automatic,CC,Doors,Weight
0,13500,23,46986,Diesel,90,1,0,2000,3,1165
1,13750,23,72937,Diesel,90,1,0,2000,3,1165
2,13950,24,41711,Diesel,90,1,0,2000,3,1165
3,14950,26,48000,Diesel,90,0,0,2000,3,1165
4,13750,30,38500,Diesel,90,0,0,2000,3,1170


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1268 entries, 0 to 1267
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      1268 non-null   int64 
 1   Age        1268 non-null   int64 
 2   KM         1268 non-null   int64 
 3   FuelType   1268 non-null   object
 4   HP         1268 non-null   int64 
 5   MetColor   1268 non-null   object
 6   Automatic  1268 non-null   object
 7   CC         1268 non-null   int64 
 8   Doors      1268 non-null   int64 
 9   Weight     1268 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 99.2+ KB


In [46]:
sp1 = df['Price'].loc[df['MetColor'].eq('0')]
sp2 = df['Price'].loc[df['MetColor'].eq('1')]

In [56]:
sp1.sort_values()

24       4350
225      4450
902      5600
891      5740
880      5750
        ...  
3       14950
177     14950
174     14950
1264    15500
1263    15950
Name: Price, Length: 428, dtype: int64

In [50]:
print(sp1.mean())
print(sp2.mean())

9466.672897196262
9814.132142857143


In [51]:
df.groupby(by = 'MetColor')['Price'].mean()

MetColor
0   9466.6729
1   9814.1321
Name: Price, dtype: float64

In [52]:
pd.pivot_table(data = df, values = 'Price', index = 'MetColor', aggfunc = np.mean)

,Price
MetColor,
0,9466.6729
1,9814.1321


In [53]:
pg.ttest(x = sp1, y = sp2, correction = False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.8918,1266,two-sided,0.0039,"[-583.18, -111.73]",0.1717,4.077,0.8237


In [54]:
pg.ttest(x = sp1, y = sp2, correction = True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.9836,935.8931,two-sided,0.0029,"[-576.0, -118.92]",0.1717,5.313,0.8237


In [57]:
pg.mwu(x = sp1, y = sp2)

,U-val,alternative,p-val,RBC,CLES
MWU,166376.0000,two-sided,0.0298,0.0745,0.4628


In [58]:
pg.normality(data = df, dv = 'Price', group = 'Automatic')

,W,pval,normal
Automatic,,,
0,0.9815,0.0000,False
1,0.9484,0.0060,False


In [59]:
pg.homoscedasticity(data = df, dv = 'Price', group = 'Automatic')

,W,pval,equal_var
levene,1.0930,0.2960,True


In [60]:
sp1 = df['Price'].loc[df['Automatic'].eq('0')]
sp2 = df['Price'].loc[df['Automatic'].eq('1')]

In [61]:
print(sp1.mean())
print(sp2.mean())

9676.575125208681
10043.857142857143


In [62]:
pg.ttest(x = sp1, y = sp2, correction = False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.4727,1266,two-sided,0.1411,"[-856.55, 121.99]",0.1811,0.377,0.3129


In [63]:
pg.ttest(x = sp1, y = sp2, correction = True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.5677,78.5710,two-sided,0.1210,"[-833.65, 99.08]",0.1811,0.433,0.3129


In [64]:
pg.mwu(x = sp1, y = sp2)

,U-val,alternative,p-val,RBC,CLES
MWU,37216.5000,two-sided,0.1132,0.1124,0.4438


In [65]:
sp1.count()

1198

In [66]:
sp2.count()

70

In [67]:
sp3 = pd.concat(objs = [sp2, sp2, sp2, sp2], axis = 0, ignore_index = True)

In [68]:
pg.ttest(x = sp1, y = sp3, correction = False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.7552,1476,two-sided,0.0059,"[-628.77, -105.79]",0.1829,3.083,0.7862


In [69]:
df1 = pd.read_csv('https://bit.ly/sample_ttest')

In [70]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      30 non-null     int64  
 1   before  30 non-null     float64
 2   after   30 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 852.0 bytes


In [72]:
df1.head()

,id,before,after
0,1,88.4096,89.0478
1,2,84.7401,83.7963
2,3,106.9252,105.7830
3,4,93.2767,89.9656
4,5,104.5756,103.4107


In [73]:
df1.describe()

,id,before,after
count,30.0000,30.0000,30.0000
mean,15.5000,86.3314,85.3251
std,8.8034,9.9181,9.7795
min,1.0000,67.5036,65.4629
25%,8.2500,80.1236,79.9981
50%,15.5000,86.6179,86.1490
75%,22.7500,91.6902,90.2599
max,30.0000,106.9252,105.7830


In [75]:
stats.shapiro(df1['before'])[1]

0.9512473940849304

In [76]:
pg.ttest(x = df1['before'], y = 90, alternative = 'two.sided')

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.0260,29,two-sided,0.0521,"[82.63, 90.03]",0.3699,1.159,0.4995


In [77]:
pg.ttest(x = df1['before'], y = 90, alternative = 'less')

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.0260,29,less,0.0260,"[-inf, 89.41]",0.3699,2.318,0.6306


In [78]:
df1.apply(func = lambda x: stats.shapiro(x)[1])

id       0.2662
before   0.9512
after    0.9396
dtype: float64

In [79]:
pg.ttest(x = df1['before'], y = df1['after'], paired = True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.5367,29,two-sided,0.0000,"[0.63, 1.38]",0.1022,3529.131,0.0842


In [80]:
pg.ttest(x = df1['before'], y = df1['after'], paired = True, alternative = 'greater')

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.5367,29,greater,0.0000,"[0.7, inf]",0.1022,7058.261,0.1361


In [82]:
df2 = df1.melt(id_vars = 'id')
df2.head()

,id,variable,value
0,1,before,88.4096
1,2,before,84.7401
2,3,before,106.9252
3,4,before,93.2767
4,5,before,104.5756


In [83]:
pg.normality(data = df2, dv = 'value', group = 'variable')

,W,pval,normal
variable,,,
before,0.9859,0.9512,True
after,0.9851,0.9396,True


In [84]:
pg.normality(data = df, dv = 'Price', group = 'FuelType')

,W,pval,normal
FuelType,,,
Diesel,0.9560,0.0004,False
Petrol,0.9782,0.0000,False
CNG,0.9524,0.4957,True


In [85]:
pg.homoscedasticity(data = df, dv = 'Price', group = 'FuelType')

,W,pval,equal_var
levene,6.4979,0.0016,False


In [86]:
pg.anova(data = df, dv = 'Price', between = 'FuelType')

,Source,ddof1,ddof2,F,p-unc,np2
0,FuelType,2,1265,3.9827,0.0189,0.0063


In [87]:
pg.welch_anova(data = df, dv = 'Price', between = 'FuelType')

,Source,ddof1,ddof2,F,p-unc,np2
0,FuelType,2,38.9415,2.6120,0.0862,0.0063


In [88]:
df['FuelType'].value_counts().sort_index()

FuelType
CNG         17
Diesel     126
Petrol    1125
Name: count, dtype: int64

In [89]:
pg.kruskal(data = df, dv = 'Price', between = 'FuelType')

,Source,ddof1,H,p-unc
Kruskal,FuelType,2,10.4747,0.0053


In [90]:
import scikit_posthocs as sp

In [91]:
sp.posthoc_tukey(a = df, val_col = 'Price', group_col = 'FuelType')

,Diesel,Petrol,CNG
Diesel,1.0000,0.0159,0.9000
Petrol,0.0159,1.0000,0.7593
CNG,0.9000,0.7593,1.0000


In [93]:
sp.posthoc_tukey_hsd(x = df['Price'], g = df['FuelType'])

,CNG,Diesel,Petrol
CNG,1,0,0
Diesel,0,1,1
Petrol,0,1,1


In [94]:
sp.posthoc_scheffe(a = df, val_col = 'Price', group_col = 'FuelType')

,Diesel,Petrol,CNG
Diesel,1.0000,0.0221,0.9339
Petrol,0.0221,1.0000,0.7980
CNG,0.9339,0.7980,1.0000


In [95]:
sp.posthoc_tamhane(a = df, val_col = 'Price', group_col = 'FuelType')

,Diesel,Petrol,CNG
Diesel,1.0000,0.0757,0.9874
Petrol,0.0757,1.0000,0.9318
CNG,0.9874,0.9318,1.0000


In [96]:
sp.posthoc_nemenyi(a = df, val_col = 'Price', group_col = 'FuelType')

,CNG,Diesel,Petrol
CNG,1.0000,0.9543,0.6756
Diesel,0.9543,1.0000,0.0071
Petrol,0.6756,0.0071,1.0000


In [97]:
df2 = df[df['FuelType'].ne('CNG')]

In [98]:
pg.homoscedasticity(data = df2, dv = 'Price', group = 'FuelType')

,W,pval,equal_var
levene,12.3698,0.0005,False


In [100]:
sp1 = df2['Price'][df2['FuelType'].eq('Diesel')]
sp2 = df2['Price'][df2['FuelType'].eq('Petrol')]

In [101]:
print(sp1.mean())
print(sp2.mean())

9227.70634920635
9753.56088888889


In [102]:
pg.ttest(x = sp1, y = sp2, correction = True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.2516,142.0118,two-sided,0.0259,"[-987.54, -64.17]",0.2606,1.21,0.7917


In [103]:
df2 = pd.read_csv('https://bit.ly/sample_cross')

In [104]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Coupon    1000 non-null   object
 1   Purchase  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [105]:
df2.head()

,Coupon,Purchase
0,발송함,구매안함
1,발송안함,구매안함
2,발송함,구매안함
3,발송안함,구매안함
4,발송함,구매안함


In [106]:
df2.describe(include = object)

,Coupon,Purchase
count,1000,1000
unique,2,2
top,발송함,구매안함
freq,500,900


In [111]:
pd.crosstab(
    index = df2['Coupon'], 
    columns = df2['Purchase'], 
    margins = True, 
    margins_name = '합계', 
    normalize = 'index'
)

Purchase,구매안함,구매함
Coupon,,
발송안함,0.9800,0.0200
발송함,0.8200,0.1800
합계,0.9000,0.1000


In [113]:
test = pg.chi2_independence(data = df2, x = 'Coupon', y = 'Purchase', correction = True)

In [114]:
test[0]

Purchase,구매안함,구매함
Coupon,,
발송안함,450.0000,50.0000
발송함,450.0000,50.0000


In [115]:
test[1]

Purchase,구매안함,구매함
Coupon,,
발송안함,489.5000,10.5000
발송함,410.5000,89.5000


In [116]:
test[2]

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.0000,69.3444,1.0000,0.0000,0.2633,1.0000
1,cressie-read,0.6667,71.1142,1.0000,0.0000,0.2667,1.0000
2,log-likelihood,0.0000,78.3864,1.0000,0.0000,0.2800,1.0000
3,freeman-tukey,-0.5000,88.4847,1.0000,0.0000,0.2975,1.0000
4,mod-log-likelihood,-1.0000,104.8045,1.0000,0.0000,0.3237,1.0000
5,neyman,-2.0000,173.0165,1.0000,0.0000,0.4160,1.0000


In [117]:
from statsmodels.stats.proportion import proportions_ztest

In [118]:
proportions_ztest(count = 30, nobs = 100, value = 0.2)

(2.182178902359923, 0.029096331741252257)

In [119]:
proportions_ztest(count = [100, 150], nobs = [300, 500], value = 0.0)

(0.9847319278346616, 0.3247557654026184)

In [120]:
df = df.drop(columns = ['CC', 'Automatic'])

In [121]:
df = df[df['FuelType'].ne('CNG')]

In [122]:
df = df.reset_index(drop = True)

In [123]:
joblib.dump(value = df, filename = 'Used_Cars_Price_Prep.z')

['Used_Cars_Price_Prep.z']